In [4]:
from parlai.core.params import ParlaiParser
from parlai.core.dict import DictionaryAgent
from parlai.scripts.train_model import setup_args as tm_setupargs
from parlai.scripts.train_model import TrainLoop
from parlai.core.agents import create_agent
from parlai.core.worlds import create_task
import json
import time
import os

In [5]:
SHARED = {}
def setup_interactive(shared):
    parser = tm_setupargs()
    parser.set_params(batchsize=5, beam_size=20, beam_min_n_best=10)
    SHARED['opt'] = parser.parse_args([], print_args=True)

    SHARED['opt']['task'] = 'projects.jp_dialogue.tasks.agents'
    SHARED['opt']['model'] = 'bert_ranker/bi_encoder_ranker'
#     SHARED['opt']['dict_class'] = 'projects.jp_dialogue.jp_retrieval.retrieval_agents:UniBiDictionaryAgent'
#     SHARED['opt']['dict_build_first'] = True
#     SHARED['opt']['dict_minfreq'] = 5
    SHARED['opt']['model_file'] = '/installation/~/ParlAI/data/models/jp_dialogue/something_bert'
    SHARED['opt']['no_cuda'] = True
    SHARED['opt']['history_size'] = 1
    SHARED['opt']['truncate'] = -1
    SHARED['opt']['interactive_mode'] = True
    SHARED['opt']['candidates'] = 'batch'
    SHARED['opt']['eval_candidates'] = 'batch'
    SHARED['opt']['encode_candidate_vecs'] = False
    SHARED['opt']['fixed_candidates_path'] = ''
    SHARED['opt']['fixed_candidate_vecs'] = 'reuse'
#     SHARED['opt']['scoring_func'] = 'scaled'
#     SHARED['opt']['embedding_size'] = 300
#     SHARED['opt']['h_layer_num'] = 3
#     SHARED['opt']['h_dim'] = 512
#     SHARED['opt']['h_act_func'] = 'swish'
#     SHARED['opt']['linear_dim'] = 1024
#     SHARED['opt']['n_heads'] = 12
#     SHARED['opt']['n_layers'] = 1
#     SHARED['opt']['ffn_size'] = 3072
#     SHARED['opt']['dropout'] = 0.1
#     SHARED['opt']['attention_dropout'] = 0.1
#     SHARED['opt']['relu_dropout'] = 0.0
#     SHARED['opt']['learn_positional_embeddings'] = True
#     SHARED['opt']['embeddings_scale'] = False
#     SHARED['opt']['activation'] = 'gelu'
#     SHARED['opt']['variant'] = 'xlm'
#     SHARED['opt']['output_scaling'] = 1.0
#     SHARED['opt']['eval_candidates'] = ''
#     SHARED['opt']['learningrate'] = 5e-05
#     SHARED['opt']['momentum'] = 0
#     SHARED['opt']['optimizer'] = 'sgd'
    SHARED['opt']['lr_scheduler'] ='reduceonplateau'
    SHARED['opt']['out_dim'] =768
    SHARED['opt']['add_transformer_layer']=False
    SHARED['opt']['pull_from_layer']=-1
    SHARED['opt']['bert_aggregation']='mean'
    SHARED['opt']['dict_maxexs']=0
    SHARED['opt']['train_folder']='rachel'
    # Create model and assign it to the specified task
    SHARED['world'] = TrainLoop(SHARED.get('opt'))

In [ ]:
setup_interactive(SHARED)

[ Main ParlAI Arguments: ] 
[  batchsize: 5 ]
[  datapath: d:\installation\~\parlai\data ]
[  datatype: train ]
[  download_path: d:\installation\~\parlai\downloads ]
[  hide_labels: False ]
[  image_mode: raw ]
[  init_opt: None ]
[  multitask_weights: [1] ]
[  numthreads: 1 ]
[  show_advanced_args: False ]
[  task: None ]
[ ParlAI Model Arguments: ] 
[  dict_class: None ]
[  init_model: None ]
[  model: None ]
[  model_file: None ]
[ Training Loop Arguments: ] 
[  aggregate_micro: False ]
[  dict_build_first: True ]
[  display_examples: False ]
[  eval_batchsize: None ]
[  evaltask: None ]
[  load_from_checkpoint: False ]
[  max_train_time: -1 ]
[  metrics: default ]
[  num_epochs: -1 ]
[  save_after_valid: False ]
[  save_every_n_secs: -1 ]
[  short_final_eval: False ]
[  validation_cutoff: 1.0 ]
[  validation_every_n_epochs: -1 ]
[  validation_every_n_secs: -1 ]
[  validation_max_exs: -1 ]
[  validation_metric: accuracy ]
[  validation_metric_mode: None ]
[  validation_patience: 10

 38%|██████████████████████████████▋                                                 | 169/440 [00:15<00:24, 11.25it/s]

In [ ]:
world = SHARED['world'].world

In [ ]:
batch_act = world.batch_act(0, None)

In [ ]:
world.world.get_agents()

In [ ]:
batch_act

In [ ]:
batch_obs = world.batch_observe(1, batch_act, 0)

In [ ]:
batch_obs

In [ ]:
model_agent = world.world.get_agents()[1]

In [ ]:
dict_class = model_agent.dict

In [ ]:
v =[   1, 1869,  378,    5,    9,    2]
for t in v:
    print(dict_class.ind2bi[t])

In [ ]:
batch = model_agent.batchify(batch_obs)

In [ ]:
batch

In [ ]:
cands, cand_vecs, label_inds = model_agent._build_candidates(
            batch, source='batch', mode='train'
        )

In [ ]:
label_inds

In [ ]:
cand_vecs

In [ ]:
batch.text_vec.size(0)

In [ ]:
scores = model_agent.model(batch.text_vec, batch.bi_text_vec, cand_vecs['uni'], cand_vecs['bi'])

In [ ]:
cand_vecs

In [ ]:
x_uni = batch.text_vec
x_bi = batch.bi_text_vec
y_uni = cand_vecs['uni']
y_bi = cand_vecs['bi']

In [ ]:
x_emb, y_emb = model_agent.model.encode(x_uni, x_bi, y_uni.unsqueeze(1), y_bi.unsqueeze(1))

In [ ]:
y_emb

In [ ]:
y_emb.expand(y_emb.size(0), x_uni.size(0), -1).transpose(0,1).contiguous()

In [ ]:
bsz, cand_num, seq_len = y_uni.unsqueeze(1).shape

In [ ]:
y_emb.view(bsz, cand_num, -1).expand(y_emb.size(0), x_uni.size(0), -1).transpose(0,1).contiguous()

In [ ]:
y_uni_emb = model_agent.model.y_unigram_encoder(y_uni.unsqueeze(1).view(bsz*cand_num, -1))
y_bi_emb = model_agent.model.y_bigram_encoder(y_bi.unsqueeze(1).view(bsz*cand_num, -1))

In [ ]:
y_uni_emb.shape

In [ ]:
model_agent.model.C

In [ ]:
x_emb.unsqueeze(1).expand(y_emb.view(bsz, cand_num, -1).expand(y_emb.size(0), x_uni.size(0), -1).transpose(0,1).contiguous().shape)

In [ ]:
y_emb = y_emb.expand(cand_num, bsz, -1).transpose(0,1).contiguous()

In [ ]:
y_emb.shape

In [1]:
from transformers import BertTokenizer

In [2]:
tokenizer = BertTokenizer.from_pretrained('/installation/~/ParlAI/data/models/bert_models/bert-wiki-ja/vocab.txt', tokenize_chinese_chars=False)

In [3]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize('そっち かぁ 。 どっち に しろ あめ は ふる な だ な 。'))

[None, None, 8, None, 17, 9444, None, 11, 14045, 57, 40, 57, 8]